# Data integration with ICESat-2 - Part I

```{admonition} Learning Objectives
**Goals**
- Identify and locate non-ICESat-2 data sets
- Acquiring data from the cloud or via download
- Open data in Pandas and Xarray and basic functioning of DataFrames
```
☝️ This formatting is a Jupyter Book [admonition](https://jupyterbook.org/content/content-blocks.html#notes-warnings-and-other-admonitions), that uses a custom version of Markdown called {term}`MyST`

## Needed libraries:

In [ ]:
pip install sat-search

In [ ]:
pip install boto3

In [ ]:
pip install intake-stac

## Computing environment

We'll be using the following open source Python libraries in this notebook:

In [ ]:
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl,Rectangle, basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

import ipywidgets
import datetime
import re

In [ ]:
# %matplotlib widget
import satsearch
from satsearch import Search
import geopandas as gpd
import ast
import pandas as pd
import geoviews as gv
import hvplot.pandas
from ipywidgets import interact
from IPython.display import display, Image
import intake # if you've installed intake-STAC, it will automatically import alongside intake
import xarray as xr
import matplotlib.pyplot as plt
import boto3
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
from dask.utils import SerializableLock
import os
import hvplot.xarray
import numpy as np
from pyproj import Proj, transform

# Suppress library deprecation warnings
import warnings
warnings.filterwarnings('ignore')

## Identify and acquire the ICESat2 product(s) of interest

* What is the application of this product?
* What region and resolution is needed?

#### Download ICESat-2 ATL03 data from desired region

Remember icepyx? We are going to use that again to download some ICESat-2 ATL06 data over our region of interest.

In [ ]:
import icepyx as ipx

In [ ]:
# Specifying the necessary icepyx parameters
short_name = 'ATL06'
spatial_extent = 'hackweek_kml_jakobshavan.kml' # KML polygon centered on Jakobshavan
date_range = ['2019-04-01', '2019-04-30']
rgts = ['338'] # IS-2 RGT of interest

You may notice that we specified a RGT track. As seen below, a large number of ICESat-2 overpasses occur for Jakobshavan. In the interest of time (and computer memory), we are going to look at only one of these tracks.

In [ ]:
# Show image of area of interest (data viz tutorial will get in deeper so don't explain much):
center = [69.2, -50]
zoom = 7

# Open KML file for visualizing
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
jk = gpd.read_file(spatial_extent, driver='KML')

m = Map(basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisAquaTrueColorCR, '2020-07-18'),center=center,zoom=zoom)
geo_data = GeoData(geo_dataframe = jk)

m.add_layer(geo_data)
m.add_control(LayersControl())
m

In [ ]:
# Setup the Query object
region = ipx.Query(short_name, spatial_extent, date_range, tracks=rgts)

# Show the available granules
region.avail_granules(ids=True)

Looks like we have an ICESat-2 track! Let's quickly visualize the data to ensure that there are no clouds.

In [ ]:
# Request information from OpenAltimetry
cyclemap, rgtmap = region.visualize_elevation()

rgtmap

Looks good! Now it's time to download the data.

In [ ]:
# Set Earthdata credentials
uid = 'zhfair'
email = 'zhfair@umich.edu'
region.earthdata_login(uid, email)

# Order the granules
region.order_granules()

In [ ]:
# Download the data
path = '/home/jovyan/website2022/book/tutorials/DataIntegration/'
region.download_granules(path)

In [ ]:
import h5py

# Load the ICESat-2 data. We will just look at the central beams (GT2R/L)
is2_file = 'processed_ATL06_20190420093051_03380303_005_01_full.h5'
with h5py.File(is2_file, 'r') as f:
    is2_gt2r = pd.DataFrame(data={'lat': f['gt2r/land_ice_segments/latitude'][:],
                                  'lon': f['gt2r/land_ice_segments/longitude'][:],
                                  'elev': f['gt2r/land_ice_segments/h_li'][:]})
    is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_ice_segments/latitude'][:],
                                  'lon': f['gt2l/land_ice_segments/longitude'][:],
                                  'elev': f['gt2l/land_ice_segments/h_li'][:]})

## Identify other products of interest

1) What research problems have you wanted to address that require more than one dataset?

2) What datasets have you been interested in using with IS2 to expand its use?
    * Satellite or in situ data sets?


3) What are some datasets you’ve wished you could use, but have had difficulty finding? Why? 

We will show ATM (non-AWS) and Landsat (AWS)

### Where are the other data sets stored?

* Cloud datasets in AWS 
   https://registry.opendata.aws/ 
* NASA EarthData
   https://search.earthdata.nasa.gov/search/
* ESA Copernicus Hub
   https://scihub.copernicus.eu


ATM: https://search.earthdata.nasa.gov/search/granules?p=C1000000062-NSIDC_ECS&pg[0][v]=f&pg[0][gsk]=-start_date&sb[0]=-50.56835%2C68.83371%2C-49.00139%2C69.56591&fi=ATM&as[instrument][0]=ATM&tl=1646767454.667!3!!&m=67.08956509568404!-48.11572265625!6!1!0!0%2C2 \
Sentinel-2: https://registry.opendata.aws/sentinel-2/ \
Landsat: https://registry.opendata.aws/usgs-landsat/

## Acquire non-cloud data and open: ATM data access

Why did we choose April 2019 and RGT 338? In Spring 2019, an airborne campaign called Operation IceBridge was flown across Jakobshavan as validation for ICESat-2. Onboard was the Airborne Topographic Mapper, a lidar that works at both 532 nm (like ICESat-2) and 1064 nm (near-infrared). More information about Operation IceBridge and ATM may be found here: https://nsidc.org/data/icebridge

Here, we are going to try and co-register ATM spot measurements with ICESat-2. Because both data sets are rather large, this can be computationally expensive, so we will only consider one flight track with the ATM 532 nm beam.

Operation IceBridge data is not available on the cloud, so this data was downloaded directly from NSIDC. If you are interested in using IceBridge data, NSIDC has a useful data portal here: https://nsidc.org/icebridge/portal/map

In [ ]:
# Load the ATM data into a DataFrame
atm_file = 'ILATM2_20190506_151600_smooth_nadir3seg_50pt.csv'
atm_l2 = pd.read_csv(atm_file)

atm_l2.head()

### Opening and manipulating data in Pandas

Pandas excels at helping you explore, clean, and process tabular data, such as data stored in spreadsheets or databases. In pandas, a Series is a 1-D data table and the 2-D data table is called a DataFrame, which we saw just above.

Read csv is the easiest way to open a csv data file into a pandas DataFrame. We can specify formating, data selection, indexing and much more when reading any data into a pandas DataFrame. Here I specify different headers, assign the first column as the index, and the first column as the header (even though we are renaming it, so that it doesn't get read in with data).

In [ ]:
# Load data with specific headers
headers = ['UTC', 'Lat', 'Lon',
       'Height', 'South-to-North_Slope',
       'West-to-East_Slope', 'RMS_Fit', 'Number_Measurements',
       'Number_Measurements_Removed',
       'Distance_Of_Block_To_The_Right_Of_Aircraft', 'Track_Identifier']
atm_l2 = pd.read_csv(atm_file,names=headers,index_col=0,header=0)
atm_l2.head()

Now we can explore the data and DataFrame functions

In [ ]:
atm_l2.columns

In [ ]:
atm_l2['Lat'].head()

In [ ]:
atm_l2.Lat.head()

If we want something more intuitive as an index

In [ ]:
# Add datetime for index
atm_l2 = atm_l2.reset_index()
date = pd.to_datetime(atm_file[7:15])
time = pd.to_timedelta(atm_l2.UTC, unit='s')
atm_l2['DateTime'] = date + time
atm_l2 = atm_l2.set_index('DateTime')
atm_l2.head()

Now we can easily slice data by date, month, day, etc.

In [ ]:
atm_l2['2019-05-06'].head()

Or by a range of dates

In [ ]:
atm_l2['2019-05-06':'2019-05-07']

#### Slicing

Two methods for slicing:

* .loc for label-based indexing
* .iloc for positional indexing

In [ ]:
# Use loc to slice specific indices
atm_l2.loc[['2019-05-06 15:16:09.500', '2019-05-06 15:16:09.750']]

In [ ]:
# Select indices and columns
atm_l2.loc[['2019-05-06 15:16:09.500', '2019-05-06 15:16:09.750'], ['Lat', 'Lon', 'Height']]

In [ ]:
# Use iloc to select by row and column number
atm_l2.iloc[[0,4],[0,1,2]]

#### Statistical manipulations

In [ ]:
atm_l2.mean()

In [ ]:
atm_l2['Height'].describe()

In [ ]:
# Group rows together by a column and take the mean of each group
atm_l2.groupby('Track_Identifier').mean()

In [ ]:
atm_l2.resample('30S').sum()

One alternative to using a loop to iterate over a DataFrame is to use the pandas __apply()__ method. This function acts as a map() function in Python. It takes a function as an input and applies this function to an entire DataFrame.If you are working with tabular data, you must specify an axis you want your function to act on (0 for columns; and 1 for rows). Much like the map() function, the apply() method can also be used with anonymous functions or lambda functions. Let's look at some apply() examples using baseball data.

First, you will call the .apply() method on the atm_l2 dataframe. Then use the lambda function to iterate over the rows of the dataframe. For every row, we grab the Number_Measurements and Number_Measurements_Removed columns and pass them to the calc_total function. Finally, you will specify the axis=1 to tell the .apply() method that we want to apply it on the rows instead of columns.

In [ ]:
def calc_total(a,b):
    total = a + b
    return total

atm_l2['Total_Measurements'] = atm_l2.apply(lambda row: calc_total(row['Number_Measurements'], row['Number_Measurements_Removed']), axis=1)
atm_l2.head()

## Exercise

Try making apply work with this new function to create a new column, 'Distance_to_Jakobshavn', by using the Lat and Lon as inputs.

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def distance(a,b):
    '''
    Calculate distance between a set point and a lat lon pair
    a = lat
    b = lon
    '''
    
    jlat,jlon = 69.2330, -49.2434
    
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(jlat)
    lon1 = radians(jlon)
    lat2 = radians(a)
    lon2 = radians(b)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

atm_l2['Distance_to_Jakobshavn'] 
# = atm_l2.apply(lambda row: calc_distance(row['Lat'], row['Lon']), axis=1)
# atm_l2.head()

Reset index to start fresh for multi-indexing

In [ ]:
atm_l2 = atm_l2.reset_index()
atm_l2.head()

#### Multi-indexing

In [ ]:
atm_l2 = atm_l2.set_index(['DateTime','Track_Identifier']).sort_index()
atm_l2

To clear up some terminology, the levels of a MultiIndex are the former column names (UTC, Track_Identifier). The labels are the actual values in a level, (54969.50, 0-?, ...). Levels can be referred to by name or position, with 0 being the outermost level.

Slicing the outermost index level is pretty easy, we just use our regular .loc[row_indexer, column_indexer]. We'll select the columns Lat and Long where the UTC was ---.

In [ ]:
atm_l2.loc[['2019-05-06 15:16:09.500','2019-05-06 15:16:09.750'],['Lat','Lon']]

If you wanted to select the rows whose track was 0 or 1, .loc wants [row_indexer, column_indexer] so let's wrap the two elements of our row indexer (the list of UTCs and the Tracks) in a tuple to make it a single unit:

In [ ]:
atm_l2.loc[(['2019-05-06 15:16:09.500','2019-05-06 15:16:09.750'],[0,1]),['Lat','Lon']]

Now you want info from any UTC but for the specific tracks again (0,1). Below the : indicates all labels in this level.

In [ ]:
atm_l2.loc[pd.IndexSlice[:,[0,1]], ['Lat','Lon']]

Many attributes and functions to explore with pandas

In [ ]:
atm_l2. 

## Co-register ICESat-2 with ATM data

In [ ]:
# Load the ATM data into a DataFrame
atm_file = 'ILATM2_20190506_151600_smooth_nadir3seg_50pt.csv'
atm_l2 = pd.read_csv(atm_file)

atm_l2.head()

The ATM L2 file contains plenty of information, including surface height estimates and slope of the local topography. It also contains a track identifier - ATM takes measurements from multiple parts of the aircraft, namely starboard, port, and nadir. To keep things simple, we will filter the DataFrame to only look at the nadir track (Track_Identifier = 0).

In [ ]:
atm_l2 = atm_l2[atm_l2['Track_Identifier']==0]

# Change the longitudes to be consistent with ICESat-2
atm_l2['Longitude(deg)'] -= 360

print(atm_l2.size)

Let's take a quick look at where ATM is relative to ICESat-2...

In [ ]:
# Subset the ICESat-2 data to the ATM latitudes
is2_gt2r = is2_gt2r[(is2_gt2r['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2r['lat']>atm_l2['Latitude(deg)'].min())]
is2_gt2l = is2_gt2l[(is2_gt2l['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2l['lat']>atm_l2['Latitude(deg)'].min())]


# Set up a map with the flight tracks as overlays
from ipyleaflet import Map, basemaps, basemap_to_tiles, Polyline

m = Map(
    basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),
    center=(69.25, 310.35-360),
    zoom=10
)

gt2r_line = Polyline(
    locations=[
        [is2_gt2r['lat'].min(), is2_gt2r['lon'].max()],
        [is2_gt2r['lat'].max(), is2_gt2r['lon'].min()]
    ],
    color="green" ,
    fill=False
)
m.add_layer(gt2r_line)

gt2l_line = Polyline(
    locations=[
        [is2_gt2l['lat'].min(), is2_gt2l['lon'].max()],
        [is2_gt2l['lat'].max(), is2_gt2l['lon'].min()]
    ],
    color="green" ,
    fill=False
)
m.add_layer(gt2l_line)

atm_line = Polyline(
    locations=[
        [atm_l2['Latitude(deg)'].min(), atm_l2['Longitude(deg)'].max()],
        [atm_l2['Latitude(deg)'].max(), atm_l2['Longitude(deg)'].min()]
    ],
    color="orange" ,
    fill=False
)
m.add_layer(atm_line)

m

Looks like ATM aligns very closely with the left beam (GT2L), so hopefully the two beams will agree. The terrain over this region is quite rough, so we may expect some differences between ATM and GT2R. ICESat-2 also flew over Jakobshavan 16 days before ATM, so there might be slight differences due to ice movement.

Let's take a look at how the data sets compare.

In [ ]:
# Subset the IS-2 data to the max/min latitude of ATM
#is2_gt2r = is2_gt2r[(is2_gt2r['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2r['lat']>atm_l2['Latitude(deg)'].min())]
#is2_gt2l = is2_gt2l[(is2_gt2l['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2l['lat']>atm_l2['Latitude(deg)'].min())]

print(is2_gt2r.size)
print(is2_gt2l.size)

plt.plot(is2_gt2r['lat'], is2_gt2r['elev'], label='gt2r')
plt.plot(is2_gt2l['lat'], is2_gt2l['elev'], label='gt2l')
plt.plot(atm_l2['Latitude(deg)'], atm_l2['WGS84_Ellipsoid_Height(m)'], label='atm')
plt.xlabel('latitude')
plt.ylabel('elevation [m]')
plt.xlim([69.185, 69.275])
plt.ylim([100, 550])
plt.legend()
plt.show()

The comparison is pretty good! Sure enough, ATM appears to agree very well with GT2L! The southern reaches of the track have some disagreement, but we can also see that this region has some rough terrain that could cause some errors. Let's see if we can perform a bias assessment of GT2L relative to ATM now...

In [ ]:
# Downscale the ATM data to be of equal size to the IS-2 data
from scipy.interpolate import splrep,splev

spl = splrep(is2_gt2l['lat'], is2_gt2l['elev'], s=0)
is2_spl = splev(atm_l2['Latitude(deg)'], spl, der=0)

# Calculate GT2L bias and add it to the DataFrame
atm_l2['bias'] = atm_l2['WGS84_Ellipsoid_Height(m)'] - is2_spl

# Plot the bias curve
plt.plot(is2_gt2l['lat'], is2_gt2l['bias'])
#plt.plot(atm_l2['Latitude(deg)'], atm_l2['WGS84_Ellipsoid_Height(m)'])
#plt.plot(atm_l2['Latitude(deg)'], is2_spl)
plt.xlabel('latitude')
plt.ylabel('bias [m]')
plt.xlim([69.2, 69.26])
plt.ylim([-40, 35])
plt.show()

print(atm_l2['bias'].mean())
print(atm_l2['bias'].std())

It seems like there is a fair amount of variability in the area. Still, these values are fairly close to each other!

## Download (Landsat) images from the cloud:

An Amazon Web Services (AWS) account is required to directly access Landsat data in
the cloud housed within the usgs-landsat S3 requester pays bucket. Users can also find
all objects within the Landsat record by using either the SpatioTemporal Asset Catalog
(STAC) Browser (s3) or Satellite (SAT) Application Programing Interface (API) (https).

Scene location - https://landsat-pds.s3.amazonaws.com/c1/L8/202/025/LC08_L1TP_202025_20190420_20190507_01_T1/ \
Scene band url for download - https://landsat-pds.s3.amazonaws.com/c1/L8/202/025/LC08_L1TP_202025_20190420_20190507_01_T1/LC08_L1TP_202025_20190420_20190507_01_T1_B4.TIF

#### Using data in the cloud from S3 buckets:

Landsat S3 User Manual: https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-2032-Landsat-Commercial-Cloud-Direct-Access-Users-Guide-v2.pdf.pdf

Scene band url for access - s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/202/025/LC08_L1TP_202025_20190420_20190507_01_T1/LC08_L1TP_202025_20190420_20190507_01_T1_B4.TIF

## Find and open Landsat images in the cloud

But what if you want to search for Landsat 8 data over an area of interest? Browsing through lists of files is cumbersome. 

How can data formats enable high-speed access?

### Web Object-based Storage (e.g., Simple Storage Service)
Data storage architecture for handling large amounts of unstructured data (like geospatial datasets)
* Inexpensive storage, BUT...
* ...access is via HTTP1, not random disk reads
* Does include HTTP “range-get” to retrieve a specific byte range

### Cloud optimized approaches
* Organize data as an aggregation of small, independently retrievable objects (e.g., zarr, HDF2, Geotiff in the Cloud)
* Allow access to pieces of large objects (e.g., Cloud-Optimized GeoTIFF3,OPeNDAP4 in the Cloud)
 
We will be working with Cloud Optimized GeoTIFF (COG). A COG is a GeoTIFF file with an internal organization that enables more efficient workflows in the cloud environment.  It does this by leveraging the ability of clients issuing ​HTTP GET range requests to ask for just the parts of a file they need instead of having to open the entire image or data set.

To explore and access COG's easily we will use a SpatioTemporal Asset Catalog (STAC). The STAC specification provides a common, machine-readable (JSON) format for describing a wide range of geospatial datasets. STAC’s goal is to make it easier to index and discover any geospatial dataset that can be described by a spatial extent and time. STAC is the way geospatial asset metadata is structured and queried and it makes querrying S3 buckets easier. Learn more here: https://github.com/radiantearth/stac-spec.

In [ ]:
# Sets up credentials for acquiring images through dask/xarray
os.environ["AWS_REQUEST_PAYER"] = "requester"

# Need to set up proper credentials for acquiring data through rasterio
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
# Extract geometry bounds for Landsat spatial bbox
geom = jk.geometry[0]
print(geom.bounds)

We will search for imagery in STAC catalog using satsearch: 
https://github.com/sat-utils/sat-search

In [ ]:
# Search STAC API for Landsat images based on a bounding box, date and other metadata if desired

bbox = (-51.3229, 68.840, -48.203, 69.616) #(west, south, east, north) 

timeRange = '2019-05-06/2019-05-07'
    
results = Search.search(url='https://ibhoyw8md9.execute-api.us-west-2.amazonaws.com/prod',
                        collection='usgs-landsat/collection02/',
                        datetime=timeRange,
                        bbox=bbox,    
                        # properties=properties,
                        sort=['<datetime'])

print('%s items' % results.found())
items = results.items()

# Save search to geojson file
gjson_outfile = f'/home/jovyan/website2022/book/tutorials/DataIntegration/Landsat.geojson'
items.save(gjson_outfile)

We are given a satsearch collection of items (images)

In [ ]:
items

Load the geojson file into geopandas and inspect the items we want to collect

In [ ]:
# Load the geojson file
gf = gpd.read_file(gjson_outfile)
gf.head(2)

In [ ]:
# Plot search area of interest and frames on a map using Holoviz Libraries (more on these later)
cols = gf.loc[:,('id','landsat:wrs_path','landsat:wrs_row','geometry')]
footprints = cols.hvplot(geo=True, line_color='k', hover_cols=['landsat:wrs_path','landsat:wrs_row'], alpha=0.2, title='Landsat 8 T1',tiles='ESRI')
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * labels

#### Intake all scenes using the intake-STAC library
Intake-STAC facilitates discovering, exploring, and loading spatio-temporal datasets.

Intake-STAC provides Intake Drivers for SpatioTemporal Asset Catalogs (STAC). By combining Intake and sat-stac, Intake-STAC provides a simple toolkit for working with STAC catalogs and for loading STAC assets as Xarray objects.

In [ ]:
catalog = intake.open_stac_item_collection(items)
list(catalog)

\
Let's explore the metadata and keys for the first scene

In [ ]:
sceneids = list(catalog)
item0 = catalog[sceneids[0]]
# item0.metadata
for keys in item0.keys():
    print (keys)

We can explore the metadata for any of these:

In [ ]:
item0['blue'].metadata

In [ ]:
# This is the url needed to grab data from the S3 bucket
# Using the satsearch catalog:
items[0].asset('blue')['alternate']['s3']['href'] # can use item asset name or title (blue or B2)

# Using the intake-STAC catalog
item0.blue.metadata['alternate']['s3']['href'] # must use item asset name (blue)

# Should I keep next cell?

In [ ]:
# Retrieve first scene using rio
item_url = item0.blue.metadata['alternate']['s3']['href']

# Read and plot with grid coordinates 
with rio.Env(aws_session):
    with rio.open(item_url) as src:
        fig, ax = plt.subplots(figsize=(9,8))
        
        # To plot
        show(src,1)
        
        # To open data into a numpy array
        profile = src.profile
        arr = src.read(1)

### Manipulating data in Xarray
__Tasha__

Pandas and Xarray have very similar structures and ways of manipulating data, but Pandas excells with 2-D data and Xarray is ideal for more dimensions. Xarray introduces labels in the form of dimensions, coordinates and attributes on top of Pandas-like DataFrames.

Xarray doesn’t just keep track of labels on arrays – it uses them to provide a powerful and concise interface.

Read in 3 bands of a scene to xarray

In [ ]:
sceneid = catalog[sceneids[0]]
print (sceneid.name)

band_names = ['red','green','blue']

bands = []

# Construct xarray for scene
for band_name in band_names:
    # Specify chunk size (x,y), Landsat COG is in 512 chunks so can use this or a multiple
    band = sceneid[band_name](chunks=dict(band=1, x=512, y=512),urlpath=sceneid[band_name].metadata['alternate']['s3']['href']).to_dask()
    band['band'] = [band_name]
    bands.append(band)
scene = xr.concat(bands, dim='band')
scene

We can choose not to specify chunk sizes and have everything read as one chunk. Though, when we load larger sets of imagery, like this one, we can change these chunk sizes to use dask. It’s best to align dask chunks with the way image chunks (typically called “tiles” are stored on disk or cloud storage buckets. The landsat data is stored on AWS S3 in a tiled Geotiff format where tiles are 512x512, so we should pick some multiple of that, and typically aim for chunk sizes of ~100Mb (although this is subjective). You can read more about dask chunks here: https://docs.dask.org/en/latest/array-best-practices.html

Also check out this documentation about the Cloud-optimized Geotiff format, it is an excellent choice for putting satellite raster data on Cloud storage: https://www.cogeo.org/

Similarly as with pandas, we can explore variables easily. This time we can work with coordinates (equivalent to indices in pandas):

In [ ]:
scene.x

Or keep track of arbitrary metadata (called attributes) in the form of a Python dictionary: 

In [ ]:
scene.attrs

In [ ]:
scene.crs

We can apply operations over dimensions by name:

In [ ]:
scene.sel(band='blue')

Mathematical operations (e.g., x - y) vectorize across multiple dimensions (array broadcasting) based on dimension names, not shape.

In [ ]:
scene.sel(band='blue').sum()#.values

Easily use the split-apply-combine paradigm with groupby:

In [ ]:
scene.groupby('band').mean()

Database-like alignment based on coordinate labels that smoothly handles missing values:

The N-dimensional nature of xarray’s data structures makes it suitable for dealing with multi-dimensional scientific data, and its use of dimension names instead of axis labels (dim='time' instead of axis=0) makes such arrays much more manageable than the raw numpy ndarray.


\
Now let's open all the bands and multiple days together into an xarray. This will be a more complex xarray with an extra `'time'` dimension.

In [ ]:
sceneids = list(catalog)
sceneids

\
Let's create the time variable first for the xarray time dimension

In [ ]:
# Create time variable for time dim
time_var = xr.Variable('time',gf.loc[gf.id.isin([sceneid for sceneid in sceneids if sceneid.endswith('_T1')])]['datetime'])

Now we will search and collect band names for grabbing each desired band. We will just grab the bands that have 30 m pixels. 

In [ ]:
band_names = []

# Get band names
sceneid = catalog[sceneids[0]]
for k in sceneid.keys():
    M = getattr(sceneid, k).metadata
    if 'eo:bands' in M:
        resol = M['eo:bands'][0]['gsd']
        print(k, resol)
        if resol >= 30: 
            band_names.append(k)
            
# Add qa band
band_names.append('qa_pixel')

In [ ]:
# Import to xarray
# In xarray dataframe nans are in locations where concat of multiple scenes has expanded the grid

scenes = []

for sceneid in sceneids:
    if sceneid.endswith('_T1'):
        sceneid = catalog[sceneid]
        print (sceneid.name)
        
        bands = []
        
        # Construct xarray for scene
        for band_name in band_names:
            band = sceneid[band_name](chunks=dict(band=1, x=2048, y=2048),urlpath=sceneid[band_name].metadata['alternate']['s3']['href']).to_dask() # Specify chunk size, landsat is prob in 512 chunks so used multiple
            band['band'] = [band_name]
            bands.append(band)
        scene = xr.concat(bands, dim='band')
        scenes.append(scene)

# Concatenate scenes with time variable ***This is the slowest
ls_scenes = xr.concat(scenes, dim=time_var)

pix = ls_scenes.transform[0]
epsg = ls_scenes.crs

ls_scenes

We now have 2 Landsat scenes with all of the bands we are interested in stored in an xarray. 

We can also easily subset and visualize:

In [ ]:
sbands = ['blue', 'nir08', 'swir16','cirrus', 'lwir11']
sub_box = 1

# for t in thw.time.values:
t = ls_scenes.time.values[0]

# Further subset to polynya location if desired 
pulx = 500000
puly = 7650000
plrx = 600000
plry = 7550000

image = ls_scenes.sel(time=t,band=sbands,y=slice(plry,puly),x=slice(pulx,plrx))

# For use at the end for coordinates
pol_y = image.y
pol_x = image.x

image = np.array(image.where(image.notnull(),0))
image = np.moveaxis(image, 0, -1)

n_band = image.shape[2]
print (t)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.imshow(image[:,:,0])

## Summary

You have seen how we can search for non-ICESat_2 datasets, open them into pandas and xarray dataframes, and manipulate/explore the dataframes.
 

```{note}
You may have noticed Jupyter Book adds some extra formatting features that do not necessarily render as you might expect when *executing* a noteook in Jupyter Lab. This "admonition" note is one such example.
```

:::{warning}
Jupyter Book is very particular about [Markdown header ordering](https://jupyterbook.org/structure/sections-headers.html?highlight=headers#how-headers-and-sections-map-onto-to-book-structure) to automatically create table of contents on the website. In this tutorial we are careful to use a single main header (#) and sequential subheaders (#, ##, ###, etc.)
:::

## References

To further explore the topics of this tutorial see the following detailed documentation:

* [Jupyter Book rendering of .ipynb notebooks](https://jupyterbook.org/file-types/notebooks.html)
* [Jupyter Book guide on writing narrative content](https://jupyterbook.org/content/index.html)
* [ipyleaflet documentation](https://ipyleaflet.readthedocs.io)